In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore')

# data
import pandas as pd
import numpy as np
import random as rnd
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from collections import Counter

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
import tensorflow as tf

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [3]:
origin_df = pd.read_csv('bdb.csv', low_memory = False)
# origin_df.head(60)

In [4]:
origin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682154 entries, 0 to 682153
Data columns (total 49 columns):
GameId                    682154 non-null int64
PlayId                    682154 non-null int64
Team                      682154 non-null object
X                         682154 non-null float64
Y                         682154 non-null float64
S                         682154 non-null float64
A                         682154 non-null float64
Dis                       682154 non-null float64
Orientation               682131 non-null float64
Dir                       682126 non-null float64
NflId                     682154 non-null int64
DisplayName               682154 non-null object
JerseyNumber              682154 non-null int64
Season                    682154 non-null int64
YardLine                  682154 non-null int64
Quarter                   682154 non-null int64
GameClock                 682154 non-null object
PossessionTeam            682154 non-null object
Down   

In [6]:
working_df = origin_df.copy()

In [7]:
working_df.columns

Index(['GameId', 'PlayId', 'Team', 'X', 'Y', 'S', 'A', 'Dis', 'Orientation',
       'Dir', 'NflId', 'DisplayName', 'JerseyNumber', 'Season', 'YardLine',
       'Quarter', 'GameClock', 'PossessionTeam', 'Down', 'Distance',
       'FieldPosition', 'HomeScoreBeforePlay', 'VisitorScoreBeforePlay',
       'NflIdRusher', 'OffenseFormation', 'OffensePersonnel',
       'DefendersInTheBox', 'DefensePersonnel', 'PlayDirection', 'TimeHandoff',
       'TimeSnap', 'Yards', 'PlayerHeight', 'PlayerWeight', 'PlayerBirthDate',
       'PlayerCollegeName', 'Position', 'HomeTeamAbbr', 'VisitorTeamAbbr',
       'Week', 'Stadium', 'Location', 'StadiumType', 'Turf', 'GameWeather',
       'Temperature', 'Humidity', 'WindSpeed', 'WindDirection'],
      dtype='object')

### Setup

In [9]:
working_df.drop(columns=['GameId','Dir', 'X', 'Y', 'S', 'A', 'Dis', 'Orientation','DisplayName', 
                         'JerseyNumber', 'PlayerHeight', 'PlayerWeight', 'NflId', 
                         'PlayerBirthDate', 'PlayerCollegeName', 'Position', 'Team',
                         'Stadium', 'Location', 'WindSpeed', 'WindDirection']
                , axis= 1, inplace= True)

In [10]:
working_df.drop_duplicates(subset= "PlayId", inplace= True)

In [11]:
working_df.set_index('PlayId', inplace=True)

### Season

### YardLine

### Quarter

### GameClock

In [30]:
# Because Quarter is a feature, we can just make convert this column into seconds that are left in the quarter

def GameClockSeconds(GameClock):
    time_split = GameClock.split(':')
    seconds = int(time_split[0])*60 + int(time_split[1]) 
    return seconds

working_df['GameClock'] = working_df['GameClock'].apply(GameClockSeconds)

### PossessionTeam

In [ ]:
working_df.rename(columns={'PossessionTeam':'Team'}, inplace=True)

In [16]:
# The below code fixes incorrect names and is included in the data
# cleaning module:

working_df['Team'].replace(to_replace= {'ARZ':'ARI', 'BLT':'BAL',
                                                  'CLV':'CLE', 'HST':'HOU'}, 
                                    inplace= True)

working_df['FieldPosition'].replace(to_replace= {'ARZ':'ARI', 'BLT':'BAL',
                                                  'CLV':'CLE', 'HST':'HOU'}, 
                                    inplace= True)

In [20]:
# create isHome column
working_df['isHome'] = 0

working_df.loc[working_df['Team'] == working_df['HomeTeamAbbr'], 'isHome'] = 1 
working_df.loc[working_df['Team'] == working_df['VisitorTeamAbbr'], 'isHome'] = 0 

In [21]:
# establish categorical Opponent column

working_df.loc[working_df['Team'] == working_df['HomeTeamAbbr'], 'Opponent'] = working_df['VisitorTeamAbbr']
working_df.loc[working_df['Team'] == working_df['VisitorTeamAbbr'], 'Opponent'] = working_df['HomeTeamAbbr'] 

In [27]:
working_df.drop(columns=['HomeTeamAbbr', 'VisitorTeamAbbr'], axis=1, inplace=True)

In [31]:
working_df

,Season,YardLine,Quarter,GameClock,Team,Down,YardsToFirst,FieldPosition,ballCarrier,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,Week,StadiumType,Turf,GameWeather,Temperature,Humidity,isHome,Opponent,ScoreBeforePlay,OppScoreBeforePlay
PlayId,,,,,,,,,,,,,,,,,,,,,,,,,,,
20170907000118,2017,35,1,854,NE,3,2,NE,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,1,Outdoor,Field Turf,Clear and warm,63.0,77.0,1,KC,0,0
20170907000139,2017,43,1,832,NE,1,10,NE,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:27.000Z,2017-09-08T00:44:26.000Z,3,1,Outdoor,Field Turf,Clear and warm,63.0,77.0,1,KC,0,0
20170907000189,2017,35,1,782,NE,1,10,KC,2543773,SINGLEBACK,"1 RB, 1 TE, 3 WR",7.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:45:17.000Z,2017-09-08T00:45:15.000Z,5,1,Outdoor,Field Turf,Clear and warm,63.0,77.0,1,KC,0,0
20170907000345,2017,2,1,732,NE,2,2,KC,2539663,JUMBO,"6 OL, 2 RB, 2 TE, 0 WR",9.0,"4 DL, 4 LB, 3 DB",left,2017-09-08T00:48:41.000Z,2017-09-08T00:48:39.000Z,2,1,Outdoor,Field Turf,Clear and warm,63.0,77.0,1,KC,0,0
20170907000395,2017,25,1,728,KC,1,10,KC,2557917,SHOTGUN,"1 RB, 3 TE, 1 WR",7.0,"3 DL, 2 LB, 6 DB",right,2017-09-08T00:53:14.000Z,2017-09-08T00:53:13.000Z,7,1,Outdoor,Field Turf,Clear and warm,63.0,77.0,0,NE,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191125003419,2019,46,4,587,BAL,1,10,BAL,2562407,PISTOL,"2 RB, 2 TE, 1 WR",8.0,"4 DL, 3 LB, 4 DB",left,2019-11-26T03:44:21.000Z,2019-11-26T03:44:20.000Z,1,12,Outdoor,Grass,Clear,62.0,64.0,0,LA,42,6
20191125003440,2019,47,4,543,BAL,2,9,BAL,2562407,PISTOL,"1 RB, 1 TE, 3 WR",7.0,"3 DL, 3 LB, 5 DB",left,2019-11-26T03:45:05.000Z,2019-11-26T03:45:04.000Z,1,12,Outdoor,Grass,Clear,62.0,64.0,0,LA,42,6
20191125003496,2019,13,4,484,BAL,1,10,LA,2561324,SINGLEBACK,"1 RB, 1 TE, 3 WR",8.0,"3 DL, 3 LB, 5 DB",left,2019-11-26T03:46:54.000Z,2019-11-26T03:46:53.000Z,1,12,Outdoor,Grass,Clear,62.0,64.0,0,LA,42,6


### Down

### Distance

In [24]:
working_df.rename(columns={'Distance':'YardsToFirst'}, inplace=True)

### FieldPosition

### HomeScoreBeforePlay + VisitorScoreBeforePlay

In [22]:
working_df['ScoreBeforePlay'] = 0
working_df['OppScoreBeforePlay'] = 0

In [23]:
working_df.loc[working_df['isHome'] == 1, 'ScoreBeforePlay'] = working_df['HomeScoreBeforePlay']
working_df.loc[working_df['isHome'] == 0, 'ScoreBeforePlay'] = working_df['VisitorScoreBeforePlay']

working_df.loc[working_df['isHome'] == 1, 'OppScoreBeforePlay'] = working_df['VisitorScoreBeforePlay']
working_df.loc[working_df['isHome'] == 0, 'OppScoreBeforePlay'] = working_df['HomeScoreBeforePlay']

In [26]:
working_df.drop(columns=['HomeScoreBeforePlay', 'VisitorScoreBeforePlay'], axis=1, inplace=True)

### NflIdRusher

In [29]:
working_df.rename(columns={'NflIdRusher':'ballCarrier'}, inplace=True)

### OffenseFormation

In [44]:
working_df['OffenseFormation'].value_counts()

SINGLEBACK    13624
SHOTGUN        9389
I_FORM         6225
PISTOL          979
JUMBO           677
WILDCAT          77
EMPTY            31
ACE               1
Name: OffenseFormation, dtype: int64

### OffensePersonnel

In [32]:
# FIX GROUPS

### these we can make assumption of 1 QB and 5 OL, add both:

# 1 RB, 1 TE, 3 WR                310288 -- 1 QB, 5 OL, 1 RB, 1 TE, 3 WR
# 1 RB, 2 TE, 2 WR                155232 -- 1 QB, 5 OL, 1 RB, 2 TE, 2 WR 
# 2 RB, 1 TE, 2 WR                 71764 -- 1 QB, 5 OL, 2 RB, 1 TE, 2 WR 
# 1 RB, 3 TE, 1 WR                 39556 -- 1 QB, 5 OL, 1 RB, 3 TE, 1 WR 
# 2 RB, 2 TE, 1 WR                 34958 -- 1 QB, 5 OL, 2 RB, 2 TE, 1 WR
# 2 RB, 0 TE, 3 WR                  4752 -- 1 QB, 5 OL, 2 RB, 0 TE, 3 WR 
# 1 RB, 0 TE, 4 WR                  3828 -- 1 QB, 5 OL, 1 RB, 0 TE, 4 WR
# 2 RB, 3 TE, 0 WR                  2134 -- 1 QB, 5 OL, 2 RB, 3 TE, 0 WR
# 3 RB, 1 TE, 1 WR                   726 -- 1 QB, 5 OL, 3 RB, 1 TE, 1 WR 
# 3 RB, 0 TE, 2 WR                   308 -- 1 QB, 5 OL, 3 RB, 0 TE, 2 WR 
# 0 RB, 2 TE, 3 WR                   220 -- 1 QB, 5 OL, 0 RB, 2 TE, 3 WR 
# 1 RB, 4 TE, 0 WR                   154 -- 1 QB, 5 OL, 1 RB, 4 TE, 0 WR 
# 3 RB, 2 TE, 0 WR                   132 -- 1 QB, 5 OL, 3 RB, 2 TE, 0 WR 
# 0 RB, 3 TE, 2 WR                    66 -- 1 QB, 5 OL, 0 RB, 3 TE, 2 WR 
# 0 RB, 0 TE, 5 WR                    44 -- 1 QB, 5 OL, 0 RB, 0 TE, 5 WR 
# 0 RB, 1 TE, 4 WR                  1738 -- 1 QB, 5 OL, 0 RB, 1 TE, 4 WR 

working_df['OffensePersonnel'].replace(to_replace={'1 RB, 1 TE, 3 WR' : '1 QB, 5 OL, 1 RB, 1 TE, 3 WR', 
                                                   '1 RB, 2 TE, 2 WR' : '1 QB, 5 OL, 1 RB, 2 TE, 2 WR',  
                                                   '2 RB, 1 TE, 2 WR' : '1 QB, 5 OL, 2 RB, 1 TE, 2 WR',  
                                                   '1 RB, 3 TE, 1 WR' : '1 QB, 5 OL, 1 RB, 3 TE, 1 WR',  
                                                   '2 RB, 2 TE, 1 WR' : '1 QB, 5 OL, 2 RB, 2 TE, 1 WR', 
                                                   '2 RB, 0 TE, 3 WR' : '1 QB, 5 OL, 2 RB, 0 TE, 3 WR',  
                                                   '1 RB, 0 TE, 4 WR' : '1 QB, 5 OL, 1 RB, 0 TE, 4 WR', 
                                                   '2 RB, 3 TE, 0 WR' : '1 QB, 5 OL, 2 RB, 3 TE, 0 WR', 
                                                   '3 RB, 1 TE, 1 WR' : '1 QB, 5 OL, 3 RB, 1 TE, 1 WR',  
                                                   '3 RB, 0 TE, 2 WR' : '1 QB, 5 OL, 3 RB, 0 TE, 2 WR',  
                                                   '0 RB, 2 TE, 3 WR' : '1 QB, 5 OL, 0 RB, 2 TE, 3 WR',  
                                                   '1 RB, 4 TE, 0 WR' : '1 QB, 5 OL, 1 RB, 4 TE, 0 WR',  
                                                   '3 RB, 2 TE, 0 WR' : '1 QB, 5 OL, 3 RB, 2 TE, 0 WR',  
                                                   '0 RB, 3 TE, 2 WR' : '1 QB, 5 OL, 0 RB, 3 TE, 2 WR',  
                                                   '0 RB, 0 TE, 5 WR' : '1 QB, 5 OL, 0 RB, 0 TE, 5 WR',
                                                   '0 RB, 1 TE, 4 WR' : '1 QB, 5 OL, 0 RB, 1 TE, 4 WR'
                                                  }, inplace = True)
                                       

### these we can make assumption of 1 QB and 6 OL, add QB:

# 6 OL, 3 RB, 0 TE, 1 WR              22 -- 1 QB, 6 OL, 3 RB, 0 TE, 1 WR 
# 6 OL, 0 RB, 2 TE, 2 WR              22 -- 1 QB, 6 OL, 0 RB, 2 TE, 2 WR 
# 6 OL, 2 RB, 0 TE, 2 WR            1452 -- 1 QB, 6 OL, 2 RB, 0 TE, 2 WR
# 6 OL, 1 RB, 3 TE, 0 WR            1606 -- 1 QB, 6 OL, 1 RB, 3 TE, 0 WR
# 6 OL, 1 RB, 0 TE, 3 WR            2948 -- 1 QB, 6 OL, 1 RB, 0 TE, 3 WR
# 6 OL, 2 RB, 2 TE, 0 WR            4444 -- 1 QB, 6 OL, 2 RB, 2 TE, 0 WR

# 6 OL, 2 RB, 1 TE, 1 WR            7392 -- 1 QB, 6 OL, 2 RB, 1 TE, 1 WR
# 6 OL, 1 RB, 1 TE, 2 WR           12936 -- 1 QB, 6 OL, 1 RB, 1 TE, 2 WR
# 6 OL, 1 RB, 2 TE, 1 WR           14190 -- 1 QB, 6 OL, 1 RB, 2 TE, 1 WR

working_df['OffensePersonnel'].replace(to_replace={'6 OL, 3 RB, 0 TE, 1 WR' : '1 QB, 6 OL, 3 RB, 0 TE, 1 WR', 
                                                   '6 OL, 0 RB, 2 TE, 2 WR' : '1 QB, 6 OL, 0 RB, 2 TE, 2 WR', 
                                                   '6 OL, 2 RB, 0 TE, 2 WR' : '1 QB, 6 OL, 2 RB, 0 TE, 2 WR', 
                                                   '6 OL, 1 RB, 3 TE, 0 WR' : '1 QB, 6 OL, 1 RB, 3 TE, 0 WR', 
                                                   '6 OL, 1 RB, 0 TE, 3 WR' : '1 QB, 6 OL, 1 RB, 0 TE, 3 WR',
                                                   '6 OL, 2 RB, 2 TE, 0 WR' : '1 QB, 6 OL, 1 RB, 0 TE, 3 WR', 
                                                   '6 OL, 2 RB, 1 TE, 1 WR' : '1 QB, 6 OL, 2 RB, 1 TE, 1 WR', 
                                                   '6 OL, 1 RB, 1 TE, 2 WR' : '1 QB, 6 OL, 1 RB, 1 TE, 2 WR', 
                                                   '6 OL, 1 RB, 2 TE, 1 WR' : '1 QB, 6 OL, 1 RB, 2 TE, 1 WR'
                                                  }, inplace= True)


### due to high count of plays, it is unlikely 7 ol were used. RB was probs seen as OL. 
### subtract one OL, add on RB, add QB

# 7 OL, 1 RB, 0 TE, 2 WR             682 -- 1 QB, 6 OL, 2 RB, 0 TE, 2 WR
# 7 OL, 1 RB, 2 TE, 0 WR             308 -- 1 QB, 6 OL, 2 RB, 2 TE, 0 WR
# 7 OL, 1 RB, 1 TE, 1 WR              44 -- 1 QB, 6 OL, 1 RB, 2 TE, 1 WR
# 7 OL, 2 RB, 1 TE, 0 WR              88 -- 1 QB, 6 OL, 3 RB, 1 TE, 0 WR
# 7 OL, 2 RB, 0 TE, 1 WR             220 -- 1 QB, 6 OL, 3 RB, 0 TE, 1 WR
# 7 OL, 1 RB, 2 TE, 0 WR             308 -- 1 QB, 6 OL, 2 RB, 2 TE, 0 WR

working_df['OffensePersonnel'].replace(to_replace={'7 OL, 1 RB, 0 TE, 2 WR' : '1 QB, 6 OL, 2 RB, 0 TE, 2 WR', 
                                                   '7 OL, 1 RB, 2 TE, 0 WR' : '1 QB, 6 OL, 2 RB, 2 TE, 0 WR', 
                                                   '7 OL, 1 RB, 1 TE, 1 WR' : '1 QB, 6 OL, 1 RB, 2 TE, 1 WR', 
                                                   '7 OL, 2 RB, 1 TE, 0 WR' : '1 QB, 6 OL, 3 RB, 1 TE, 0 WR',
                                                   '7 OL, 2 RB, 0 TE, 1 WR' : '1 QB, 6 OL, 3 RB, 0 TE, 1 WR',
                                                   '7 OL, 1 RB, 2 TE, 0 WR' : '1 QB, 6 OL, 2 RB, 2 TE, 0 WR'
                                                  }, inplace= True)

### the following were corrected by verifying actual game film for each play (NFL GamePass)
### play numbers are included followed by correct grouping and ind. replacement code:

# 1 RB, 1 TE, 2 WR,1 LB               66 -- 1 QB, 5 OL, 2 RB, 1 TE, 2 WR -- 20171113000112 / 20171113001385 / 20191117101942
# 1 RB, 0 TE, 3 WR,1 DB               66 -- 1 QB, 5 OL, 1 RB, 0 TE, 4 WR -- 20171105060104 / 20171112061099 / 20171112063586
# 2 QB, 2 RB, 0 TE, 2 WR              66 -- 1 QB, 5 OL, 3 RB, 0 TE, 2 WR -- 20181021012259 / 20181209042583 / 20191117063033
# 1 RB, 2 TE, 1 WR,1 DB               66 -- 1 QB, 5 OL, 1 RB, 2 TE, 2 WR -- 20181111110273 / 20181118002325 / 20181129000675
# 2 QB, 1 RB, 3 TE, 0 WR              44 -- 1 QB, 5 OL, 2 RB, 3 TE, 0 WR -- 20181028013051 / 20181028112146
# 6 OL, 1 RB, 0 TE, 2 WR,1 LB         44 -- 1 QB, 6 OL, 2 RB, 0 TE, 2 WR -- 20191117100236 / 20191117100497
# 2 RB, 2 TE, 0 WR,1 DL               44 -- 1 QB, 6 OL, 2 RB, 2 TE, 0 WR -- 20180930100710 / 20181101000953
# 7 OL, 1 RB, 1 TE, 0 WR,1 LB         44 -- 1 QB, 6 OL, 3 RB, 1 TE, 0 WR -- 20190915090924 / 20190915090949
# 2 QB, 2 RB, 2 TE, 0 WR              22 -- 1 QB, 5 OL, 3 RB, 2 TE, 0 WR -- 20181008003713
# 2 RB, 3 TE, 1 WR                    22 -- 1 QB, 5 OL, 1 RB, 3 TE, 1 WR -- 20170924102908
# 1 RB, 3 TE, 0 WR,1 DB               22 -- 1 QB, 5 OL, 1 RB, 3 TE, 1 WR -- 20171116000514
# 6 OL, 1 RB, 1 TE, 1 WR,1 LB         22 -- 1 QB, 6 OL, 2 RB, 1 TE, 1 WR -- 20191124090534
# 1 RB, 2 TE, 3 WR                    22 -- 1 QB, 5 OL, 1 RB, 2 TE, 2 WR -- 20171112080136
# 2 RB, 1 TE, 1 WR,1 DB               22 -- 1 QB, 5 OL, 2 RB, 1 TE, 2 WR -- 20181104013118
# 6 OL, 1 RB, 1 TE, 0 WR,2 DL         22 -- 1 QB, 6 OL, 3 RB, 1 TE, 0 WR -- 20181202071749
# 2 QB, 3 RB, 1 TE, 0 WR              22 -- 1 QB, 6 OL, 3 RB, 1 TE, 0 WR -- 20181021012713

working_df['OffensePersonnel'].replace(to_replace={'1 RB, 1 TE, 2 WR,1 LB' : '1 QB, 5 OL, 2 RB, 1 TE, 2 WR', 
                                                   '1 RB, 0 TE, 3 WR,1 DB' : '1 QB, 5 OL, 1 RB, 0 TE, 4 WR', 
                                                   '2 QB, 2 RB, 0 TE, 2 WR' : '1 QB, 5 OL, 3 RB, 0 TE, 2 WR', 
                                                   '1 RB, 2 TE, 1 WR,1 DB' : '1 QB, 5 OL, 1 RB, 2 TE, 2 WR', 
                                                   '2 QB, 1 RB, 3 TE, 0 WR' : '1 QB, 5 OL, 2 RB, 3 TE, 0 WR', 
                                                   '6 OL, 1 RB, 0 TE, 2 WR,1 LB' : '1 QB, 6 OL, 2 RB, 0 TE, 2 WR', 
                                                   '2 RB, 2 TE, 0 WR,1 DL' : '1 QB, 6 OL, 2 RB, 2 TE, 0 WR', 
                                                   '7 OL, 1 RB, 1 TE, 0 WR,1 LB' : '1 QB, 6 OL, 3 RB, 1 TE, 0 WR', 
                                                   '2 QB, 2 RB, 2 TE, 0 WR' : '1 QB, 5 OL, 3 RB, 2 TE, 0 WR', 
                                                   '2 RB, 3 TE, 1 WR' : '1 QB, 5 OL, 1 RB, 3 TE, 1 WR', 
                                                   '1 RB, 3 TE, 0 WR,1 DB' : '1 QB, 5 OL, 1 RB, 3 TE, 1 WR', 
                                                   '6 OL, 1 RB, 1 TE, 1 WR,1 LB' : '1 QB, 6 OL, 2 RB, 1 TE, 1 WR', 
                                                   '1 RB, 2 TE, 3 WR' : '1 QB, 5 OL, 1 RB, 2 TE, 2 WR', 
                                                   '2 RB, 1 TE, 1 WR,1 DB' : '1 QB, 5 OL, 2 RB, 1 TE, 2 WR', 
                                                   '6 OL, 1 RB, 1 TE, 0 WR,2 DL' : '1 QB, 6 OL, 3 RB, 1 TE, 0 WR', 
                                                   '2 QB, 3 RB, 1 TE, 0 WR' : '1 QB, 6 OL, 3 RB, 1 TE, 0 WR', 
                                                  }, inplace=True)


### finally, the following were adjusted using football logic
### when in doubt, an out of place player is counted as RB since this df is rushing plays
### whether Machine Vision or manual jersey number, DL are probs OL

# 1 RB, 2 TE, 1 WR,1 DL             3102 -- 1 QB, 6 OL, 1 RB, 2 TE, 1 WR
# 2 QB, 1 RB, 1 TE, 2 WR            1716 -- 1 QB, 5 OL, 2 RB, 1 TE, 2 WR
# 1 RB, 1 TE, 2 WR,1 DL              748 -- 1 QB, 6 OL, 1 RB, 1 TE, 2 WR
# 2 QB, 2 RB, 1 TE, 1 WR             550 -- 1 QB, 5 OL, 3 RB, 1 TE, 1 WR
# 1 RB, 3 TE, 0 WR,1 DL              506 -- 1 QB, 6 OL, 1 RB, 3 TE, 0 WR
# 2 QB, 1 RB, 2 TE, 1 WR             462 -- 1 QB, 5 OL, 2 RB, 2 TE, 1 WR
# 6 OL, 1 RB, 2 TE, 0 WR,1 LB        440 -- 1 QB, 6 OL, 2 RB, 2 TE, 0 WR
# 6 OL, 1 RB, 2 TE, 0 WR,1 DL        374 -- 1 QB, 6 OL, 2 RB, 2 TE, 0 WR
# 1 RB, 2 TE, 1 WR,1 LB              264 -- 1 QB, 5 OL, 2 RB, 2 TE, 1 WR
# 1 RB, 1 TE, 2 WR,1 DB              242 -- 1 QB, 5 OL, 1 RB, 1 TE, 3 WR
# 6 OL, 2 RB, 1 TE, 0 WR,1 DL        198 -- 1 QB, 6 OL, 3 RB, 1 TE, 0 WR
# 2 QB, 1 RB, 0 TE, 3 WR             198 -- 1 QB, 5 OL, 2 RB, 0 TE, 3 WR
# 2 QB, 6 OL, 1 RB, 1 TE, 1 WR       176 -- 1 QB, 6 OL, 2 RB, 1 TE, 1 WR
# 6 OL, 1 RB, 1 TE, 1 WR,1 DL        154 -- 1 QB, 6 OL, 2 RB, 1 TE, 1 WR
# 1 RB, 3 TE, 0 WR,1 LB              154 -- 1 QB, 5 OL, 2 RB, 3 TE, 0 WR

working_df['OffensePersonnel'].replace(to_replace={'1 RB, 2 TE, 1 WR,1 DL' : '1 QB, 6 OL, 1 RB, 2 TE, 1 WR', 
                                                   '2 QB, 1 RB, 1 TE, 2 WR' : '1 QB, 5 OL, 2 RB, 1 TE, 2 WR',
                                                   '1 RB, 1 TE, 2 WR,1 DL' : '1 QB, 6 OL, 1 RB, 1 TE, 2 WR', 
                                                   '2 QB, 2 RB, 1 TE, 1 WR' : '1 QB, 5 OL, 3 RB, 1 TE, 1 WR', 
                                                   '1 RB, 3 TE, 0 WR,1 DL' : '1 QB, 6 OL, 1 RB, 3 TE, 0 WR',
                                                   '2 QB, 1 RB, 2 TE, 1 WR' : '1 QB, 5 OL, 2 RB, 2 TE, 1 WR', 
                                                   '6 OL, 1 RB, 2 TE, 0 WR,1 LB' : '1 QB, 6 OL, 2 RB, 2 TE, 0 WR', 
                                                   '6 OL, 1 RB, 2 TE, 0 WR,1 DL' : '1 QB, 6 OL, 2 RB, 2 TE, 0 WR', 
                                                   '1 RB, 2 TE, 1 WR,1 LB' : '1 QB, 5 OL, 2 RB, 2 TE, 1 WR', 
                                                   '1 RB, 1 TE, 2 WR,1 DB' : '1 QB, 5 OL, 1 RB, 1 TE, 3 WR', 
                                                   '6 OL, 2 RB, 1 TE, 0 WR,1 DL' : '1 QB, 6 OL, 3 RB, 1 TE, 0 WR', 
                                                   '2 QB, 1 RB, 0 TE, 3 WR' : '1 QB, 5 OL, 2 RB, 0 TE, 3 WR', 
                                                   '2 QB, 6 OL, 1 RB, 1 TE, 1 WR' : '1 QB, 6 OL, 2 RB, 1 TE, 1 WR', 
                                                   '6 OL, 1 RB, 1 TE, 1 WR,1 DL' : '1 QB, 6 OL, 2 RB, 1 TE, 1 WR',
                                                   '1 RB, 3 TE, 0 WR,1 LB' : '1 QB, 5 OL, 2 RB, 3 TE, 0 WR'
                                                  }, inplace= True)

In [33]:
# function needs to go to lh, removed from here 

def labelRB(OffensePersonnel):
    groups = OffensePersonnel.split(',')
    for group in groups:
        if "RB" in group:
            backs = [int(x) for x in group if x.isdigit()]
            backs = backs[0]
            return backs

working_df['RB'] = working_df['OffensePersonnel'].apply(labelRB)

In [34]:
# function needs to go to lh, removed from here 

def labelTE(OffensePersonnel):
    groups = OffensePersonnel.split(',')
    for group in groups:
        if "TE" in group:
            ends = [int(x) for x in group if x.isdigit()]
            ends = ends[0]
            return ends
        
working_df['TE'] = working_df['OffensePersonnel'].apply(labelTE)

In [35]:
# function needs to go to lh, removed from here 

def labelWR(OffensePersonnel):
    groups = OffensePersonnel.split(',')
    for group in groups:
        if "WR" in group:
            receivers = [int(x) for x in group if x.isdigit()]
            receivers = receivers[0]
            return receivers

working_df['WR'] = working_df['OffensePersonnel'].apply(labelWR)

In [36]:
# function needs to go to lh, removed from here 

def labelQB(OffensePersonnel):
    groups = OffensePersonnel.split(',')
    for group in groups:
        if "QB" in group:
            passers = [int(x) for x in group if x.isdigit()]
            passers = passers[0]
            return passers

working_df['QB'] = working_df['OffensePersonnel'].apply(labelQB)

In [37]:
# function needs to go to lh, removed from here 

def labelOL(OffensePersonnel):
    groups = OffensePersonnel.split(',')
    for group in groups:
        if "OL" in group:
            linemen = [int(x) for x in group if x.isdigit()]
            linemen = linemen[0]
            return linemen

working_df['OL'] = working_df['OffensePersonnel'].apply(labelOL)

In [38]:
working_df.drop(columns=['OffensePersonnel'], axis=1, inplace=True)

In [39]:
working_df

,Season,YardLine,Quarter,GameClock,Team,Down,YardsToFirst,FieldPosition,ballCarrier,OffenseFormation,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,Week,StadiumType,Turf,GameWeather,Temperature,Humidity,isHome,Opponent,ScoreBeforePlay,OppScoreBeforePlay,RB,TE,WR,QB,OL
PlayId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20170907000118,2017,35,1,854,NE,3,2,NE,2543773,SHOTGUN,6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,1,Outdoor,Field Turf,Clear and warm,63.0,77.0,1,KC,0,0,1,1,3,1,5
20170907000139,2017,43,1,832,NE,1,10,NE,2543773,SHOTGUN,6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:27.000Z,2017-09-08T00:44:26.000Z,3,1,Outdoor,Field Turf,Clear and warm,63.0,77.0,1,KC,0,0,1,1,3,1,5
20170907000189,2017,35,1,782,NE,1,10,KC,2543773,SINGLEBACK,7.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:45:17.000Z,2017-09-08T00:45:15.000Z,5,1,Outdoor,Field Turf,Clear and warm,63.0,77.0,1,KC,0,0,1,1,3,1,5
20170907000345,2017,2,1,732,NE,2,2,KC,2539663,JUMBO,9.0,"4 DL, 4 LB, 3 DB",left,2017-09-08T00:48:41.000Z,2017-09-08T00:48:39.000Z,2,1,Outdoor,Field Turf,Clear and warm,63.0,77.0,1,KC,0,0,1,0,3,1,6
20170907000395,2017,25,1,728,KC,1,10,KC,2557917,SHOTGUN,7.0,"3 DL, 2 LB, 6 DB",right,2017-09-08T00:53:14.000Z,2017-09-08T00:53:13.000Z,7,1,Outdoor,Field Turf,Clear and warm,63.0,77.0,0,NE,0,7,1,3,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191125003419,2019,46,4,587,BAL,1,10,BAL,2562407,PISTOL,8.0,"4 DL, 3 LB, 4 DB",left,2019-11-26T03:44:21.000Z,2019-11-26T03:44:20.000Z,1,12,Outdoor,Grass,Clear,62.0,64.0,0,LA,42,6,2,2,1,1,5
20191125003440,2019,47,4,543,BAL,2,9,BAL,2562407,PISTOL,7.0,"3 DL, 3 LB, 5 DB",left,2019-11-26T03:45:05.000Z,2019-11-26T03:45:04.000Z,1,12,Outdoor,Grass,Clear,62.0,64.0,0,LA,42,6,1,1,3,1,5
20191125003496,2019,13,4,484,BAL,1,10,LA,2561324,SINGLEBACK,8.0,"3 DL, 3 LB, 5 DB",left,2019-11-26T03:46:54.000Z,2019-11-26T03:46:53.000Z,1,12,Outdoor,Grass,Clear,62.0,64.0,0,LA,42,6,1,1,3,1,5


### DefendersInTheBox

### DefensePersonnel

### PlayDirection

### TimeHandoff

### TimeSnap

### Yards

### HomeTeamAbbr

### VisitorTeamAbbr

### Week

### StadiumType

### Turf

In [ ]:
working_df['Turf'].value_counts()

In [ ]:
# simplify Turf classes
# this was done by manually verifying each playing surface brand name
turf_list = ['Field Turf', 'Artificial', 'FieldTurf', 'UBU Speed Series-S5-M',
            'A-Turf Titan', 'UBU Sports Speed S5-M', 'FieldTurf360', 'Twenty-Four/Seven Turf',
            'FieldTurf 360', 'Twenty Four/Seven Turf', 'Turf', 'Field turf', 
             'UBU-Speed Series-S5-M', 'Artifical']
grass_list = ['Grass', 'Natural Grass', 'Natural', 'Naturall Grass', 'natural grass', 'grass',
             'Natural grass']
hybrid_list = ['SISGrass', 'DD GrassMaster']

working_df.loc[working_df['Turf'].isin(turf_list), 'Turf'] = "turf"
working_df.loc[working_df['Turf'].isin(grass_list), 'Turf'] = "grass"
working_df.loc[working_df['Turf'].isin(hybrid_list), 'Turf'] = "hybrid"

### GameWeather

In [47]:
working_df['GameWeather'].value_counts()

Cloudy                                                                              6687
Sunny                                                                               6504
Partly Cloudy                                                                       2540
Clear                                                                               2533
Mostly Cloudy                                                                       1271
Rain                                                                                1190
N/A (Indoors)                                                                        741
Controlled Climate                                                                   714
Mostly Sunny                                                                         488
Fair                                                                                 453
Partly Sunny                                                                         345
Mostly sunny         

### Temp

In [45]:
# temp had 63,646 null values
working_df['Temperature'].isnull().sum()

2893

### Humidity

In [46]:
working_df['Humidity'].isnull().sum()

280